In [23]:
import sqlalchemy as db
from sqlalchemy import create_engine, inspect, MetaData, distinct, func

In [24]:
engine = db.create_engine('sqlite:///C:\\Users\\peter\\Documents\\Projekte\\Zähler\\tc_all_new.db', echo = False)

In [25]:
conn = engine.connect()

#### So komme ich an die Tabellen in der DB!

In [26]:
inspector = inspect(engine)

In [27]:
tab = inspector.get_table_names()
tab = tab[:-1]

#### Und so die Spaltennamen und Typen in einer Tabelle!

In [28]:
columns_table = inspector.get_columns('tcdata_CH_0084_01') 


for c in columns_table :
    print(c['name'], c['type'])

time TEXT
vfr11 REAL
s12 REAL
vfr21 REAL
s22 REAL
vfr1 REAL
s2 REAL


#### Es ist gar nicht so einfach, in SQLalchemy auf Tabellen einer bestehenden DB zuzugreifen. Bei Verwendung von des ORM kann "automate" zur Anwendung kommen. Ohne ORM geht es wie folgt:

In [29]:
metadata = db.MetaData(bind=engine)

In [30]:
table = db.Table('tcdata_CH_0287_02', metadata, autoload = True)

#### Voila, mit der so definierten Tabelle kann ich die Abfragen starten!

#### Wobei wir später durch alle Tabellen loopen werden

## Zunächst: Was will ich alles machen auf der DB?


<ol>
    <li>Voranalyse DB</li> 
        <ul>
         <li>Analyse, was die einzelnen Spalten liefern >> Was wird berücksichtigt?</li>  
         <li>Stichprobe: Check Werte gegen Tool Stundenten </li> 
        </ul>
    <li>Prep DB</li> 
         <ul>
         <li>Ausreisser definieren und raus? >> Oder erst in pandas? </li>
         <li>Geschwindigkeiten runden !!!</li>
         <li>Zeit ggf. auf Lokal umstellen >> Prüfen, was mit SQLalchemy geht </li>     
        </ul>
     <li>DB Auswertungen für Grafiken</li> 
         <ul>
         <li>vfr nach Stunde bzwl nach Wochentag/Ende </li>
         <li>Was tun, wenn ich pro 5min a) keinen bzw. b) mehrere Datenpunkte habe </li>
         <li>speed nach vfr !!!</li>
         <li>Staus </li> 
         <li>Die Spuren im Vergleich </li>   
        </ul>
</ol>

#### Denke, die folgenden Funktionen für das Runden der Geschwindigkeit werden nicht benötigt, da ich das Runden in Pandas machen werde.

func.round(n, x)

from sqlalchemy import update
upd = update(tablename)
val = upd.values({"column_name":"value"})
cond = val.where(tablename.c.column_name == value)

In [32]:
q = db.select(func.count(distinct(table.c.time)), func.count(table.c.time))
result = conn.execute(q)
row = result.fetchone()
row

(0, 0)

#### 1. Voranalyse DB

In [33]:
ListLength = []
ListLengthDis = []
ListNull = []

In [34]:
for t in tab:
    table = db.Table(t, metadata, autoload = True)

    ListLength.append(t[10:])
    q = db.select(func.count(table.c.time))
    result = conn.execute(q)
    row = result.fetchone()
    ListLength.append(row) 
    
    ListLengthDis.append(t[10:])
    q = db.select(func.count(distinct(table.c.time)))
    result = conn.execute(q)
    row = result.fetchone()
    ListLengthDis.append(row)    
    
    q = db.select(func.count(table.c.time)).where(table.c.vfr11 == None)
    ListNull.append(t[10:])
    result = conn.execute(q)
    row = result.fetchone()
    ListNull.append(row) 

In [35]:
print(ListLength)

['0020_01', (322473,), '0020_02', (322423,), '0020_03', (322389,), '0020_04', (322442,), '0020_05', (322407,), '0020_06', (322391,), '0020_07', (322454,), '0020_08', (322473,), '0084_01', (322249,), '0084_02', (322249,), '0084_03', (322249,), '0084_04', (322249,), '0150_01', (320870,), '0150_02', (320817,), '0240_01', (322455,), '0240_02', (322473,), '0240_03', (322394,), '0240_04', (322412,), '0240_05', (322399,), '0240_06', (322391,), '0240_07', (322448,), '0240_08', (322473,), '0243_01', (322169,), '0243_02', (321766,), '0243_03', (322060,), '0243_04', (322124,), '0274_01', (321354,), '0274_02', (321063,), '0274_03', (321315,), '0274_04', (321089,), '0274_05', (320783,), '0274_06', (103453,), '0282_01', (322364,), '0282_02', (322364,), '0282_03', (322366,), '0282_04', (322364,), '0282_05', (322364,), '0282_06', (322364,), '0286_01', (16282,), '0286_02', (16282,), '0286_03', (16282,), '0286_04', (16282,), '0286_05', (16282,), '0286_06', (179,), '0286_07', (16282,), '0286_08', (16282,

In [36]:
print(ListLengthDis)

['0020_01', (322473,), '0020_02', (322423,), '0020_03', (322389,), '0020_04', (322442,), '0020_05', (322407,), '0020_06', (322391,), '0020_07', (322454,), '0020_08', (322473,), '0084_01', (322249,), '0084_02', (322249,), '0084_03', (322249,), '0084_04', (322249,), '0150_01', (320870,), '0150_02', (320817,), '0240_01', (322455,), '0240_02', (322473,), '0240_03', (322394,), '0240_04', (322412,), '0240_05', (322399,), '0240_06', (322391,), '0240_07', (322448,), '0240_08', (322473,), '0243_01', (322169,), '0243_02', (321766,), '0243_03', (322060,), '0243_04', (322124,), '0274_01', (321354,), '0274_02', (321063,), '0274_03', (321315,), '0274_04', (321089,), '0274_05', (320783,), '0274_06', (103453,), '0282_01', (322364,), '0282_02', (322364,), '0282_03', (322366,), '0282_04', (322364,), '0282_05', (322364,), '0282_06', (322364,), '0286_01', (16282,), '0286_02', (16282,), '0286_03', (16282,), '0286_04', (16282,), '0286_05', (16282,), '0286_06', (179,), '0286_07', (16282,), '0286_08', (16282,

In [37]:
print(ListNull)

['0020_01', (0,), '0020_02', (0,), '0020_03', (0,), '0020_04', (0,), '0020_05', (0,), '0020_06', (0,), '0020_07', (0,), '0020_08', (0,), '0084_01', (0,), '0084_02', (0,), '0084_03', (0,), '0084_04', (0,), '0150_01', (0,), '0150_02', (0,), '0240_01', (0,), '0240_02', (0,), '0240_03', (0,), '0240_04', (0,), '0240_05', (0,), '0240_06', (0,), '0240_07', (0,), '0240_08', (0,), '0243_01', (0,), '0243_02', (0,), '0243_03', (0,), '0243_04', (0,), '0274_01', (0,), '0274_02', (0,), '0274_03', (0,), '0274_04', (0,), '0274_05', (0,), '0274_06', (0,), '0282_01', (0,), '0282_02', (0,), '0282_03', (0,), '0282_04', (0,), '0282_05', (0,), '0282_06', (0,), '0286_01', (0,), '0286_02', (0,), '0286_03', (0,), '0286_04', (0,), '0286_05', (0,), '0286_06', (0,), '0286_07', (0,), '0286_08', (0,), '0287_01', (0,), '0287_02', (0,), '0287_03', (0,), '0287_04', (0,), '0528_01', (0,), '0528_02', (0,), '0528_03', (0,), '0528_04', (0,), '0529_01', (0,), '0529_02', (0,), '0529_03', (0,), '0529_04', (0,), '0529_05', (0

In [38]:
ListTyp0Number = []
ListTyp1Number = []
ListTyp2Number = []
ListSpeedHigh = []

In [39]:
for t in tab:
    table = db.Table(t, metadata, autoload = True)
    
    
    q = db.select(func.count(table.c.time)).where(table.c.vfr11 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListTyp0Number.append(row) 
    q = db.select(func.count(table.c.time)).where(table.c.vfr21 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListTyp1Number.append(row) 
    q = db.select(func.count(table.c.time)).where(table.c.vfr1 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListTyp2Number.append(row)  
    
    ListSpeedHigh.append(t[7:])
    q = db.select(func.count(table.c.time)).where(table.c.s12 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 
    q = db.select(func.max(table.c.s12))
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 
    
    q = db.select(func.count(table.c.time)).where(table.c.s22 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 
    q = db.select(func.max(table.c.s22))
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 
    
    q = db.select(func.count(table.c.time)).where(table.c.s2 > 0)
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 
    q = db.select(func.max(table.c.s2))
    result = conn.execute(q)
    row = result.fetchone()
    ListSpeedHigh.append(row) 


In [40]:
print("Typ0 :", ListTyp0Number, "\n\n")
print("Typ1 :", ListTyp1Number, "\n\n")
print("Typ2 :", ListTyp2Number)

Typ0 : [(637,), (317701,), (310373,), (173527,), (288970,), (314696,), (314295,), (1043,), (301092,), (265820,), (285855,), (305886,), (279166,), (260304,), (286588,), (269880,), (316250,), (283314,), (295241,), (317394,), (307916,), (531,), (312126,), (275015,), (282402,), (313101,), (316909,), (315509,), (286112,), (279646,), (314353,), (102537,), (225049,), (289705,), (218656,), (259681,), (310535,), (300115,), (37,), (16039,), (16186,), (15088,), (15620,), (179,), (16010,), (13801,), (0,), (0,), (0,), (0,), (272356,), (190466,), (189379,), (283260,), (153730,), (204374,), (167925,), (167279,), (274318,), (281514,), (282626,), (294253,), (282195,), (181846,), (226422,), (269630,), (176371,), (69826,), (237512,), (252860,), (194717,), (273465,), (272821,), (267788,), (221620,)] 


Typ1 : [(27,), (207771,), (61326,), (184,), (291,), (54451,), (203788,), (140,), (151271,), (1012,), (3312,), (152128,), (143574,), (148954,), (84528,), (29525,), (199034,), (8116,), (11031,), (199239,), (8

In [41]:
print(ListSpeedHigh)

['CH_0020_01', (637,), (144.0,), (27,), (91.0,), (0,), (0.0,), 'CH_0020_02', (317701,), (144.0,), (207771,), (142.0,), (0,), (0.0,), 'CH_0020_03', (310373,), (144.0,), (61326,), (144.0,), (0,), (0.0,), 'CH_0020_04', (173527,), (144.0,), (184,), (144.0,), (0,), (0.0,), 'CH_0020_05', (288970,), (144.0,), (291,), (118.0,), (0,), (0.0,), 'CH_0020_06', (314696,), (144.0,), (54451,), (118.0,), (0,), (0.0,), 'CH_0020_07', (314295,), (144.0,), (203788,), (119.0,), (0,), (0.0,), 'CH_0020_08', (1043,), (144.0,), (140,), (88.0,), (0,), (0.0,), 'CH_0084_01', (301090,), (144.0,), (151271,), (113.0,), (0,), (0.0,), 'CH_0084_02', (265810,), (144.0,), (1012,), (118.0,), (0,), (0.0,), 'CH_0084_03', (285854,), (144.0,), (3312,), (144.0,), (0,), (0.0,), 'CH_0084_04', (305885,), (144.0,), (152128,), (144.0,), (0,), (0.0,), 'CH_0150_01', (279166,), (210.0,), (143574,), (180.0,), (0,), (0.0,), 'CH_0150_02', (260304,), (126.0,), (148954,), (194.0,), (0,), (0.0,), 'CH_0240_01', (286588,), (144.0,), (84528,), 